# Private School, Public Declines? The Effect of Private School Creation on Public School Outcomes

## Research Question

As the Black Lives Matter movement swept through the nation over the summer, new questions arose in my high school alumni group chat. While I didn't attend a "private" school perse, my high school requires an application and is usually filled with high-income students with very few students of color. Against a more racially and politically charged backdrop, the Facebook group grapped with the issue of how to ensure that our elite magnet school was equitable. Ammendments to admission critera were proposed, as were greater outreach programs and quota systems. However for me, a deeper question lurked in the background: should my elite, priviledged high school--and other schools like it--even exist? 

Of course, this larger question has layers of ethical reasoning alongside plenty of unanswered empirical conundrums. In this research project I endeavor to answer one simple question: **how does the presence (and density) of private schools affect the outcomes of that county's public schools?** 

## Data Description

### Data Overview

To approach this question, I assemble a dataset of all counties in the United States with information about educational outcomes and the presence of private schools by merging information from the [Stanford Educational Data Archive (Version 4.0)](https://edopportunity.org/get-the-data/seda-archive-downloads/#documentation-4) and [Homeland Infrastructure Foundation-Level Data Private Schools](https://hifld-geoplatform.opendata.arcgis.com/datasets/private-schools?geometry=77.758%2C23.941%2C52.270%2C64.949). The Stanford Educational Data Archive (SEDA) provides county level data on achievement of students in public schools broken down by racial, gender, and socioeconomic groups for easy comparison. The Homeland Infrastructure Foundation-Level Data Private Schools dataset (HIFLD) provides a list of all private schools in the United States, with information about their enrollment and which county they belong to. I construct a count of private schools and their students in each county from the HIFLD dataset and merge that with the SEDA dataset to provide information on the presence of private schools in each county.

 ## Data Cleaning

### Part 1: Aggregating HIFLD Private School Data to a County-Level Data

In [1]:
# loads libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

1. I download the complete data from [Homeland Infrastructure Foundation-Level Data Private Schools](https://hifld-geoplatform.opendata.arcgis.com/datasets/private-schools?geometry=77.758%2C23.941%2C52.270%2C64.949) website. Their data takes the form of a csv file, which I load using Pandas.

In [14]:
# reads in the csv from HIFLD
hilfd_raw_path = 'rawdata/hifld_Private_Schools.csv'
hilfd = pd.read_csv(hilfd_raw_path)
hilfd.head()

,X,Y,FID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,START_GRAD,END_GRADE,FT_TEACHER,SHELTER_ID
0,-9.751675e+06,3.556465e+06,1,00000634,ST BENEDICT CATHOLIC SCHOOL,12786 ILLINOIS ST,ELBERTA,AL,36530,NOT AVAILABLE,...,2009/12/10 00:00:00,IMAGERY/OTHER,2009/12/31 00:00:00,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,151,2,13,12,NOT AVAILABLE
1,-9.759623e+06,3.549493e+06,2,A9500035,VICTORY CHRISTIAN ACADEMY,20511 COUNTY ROAD 12,FOLEY,AL,36535,NOT AVAILABLE,...,2009/12/10 00:00:00,IMAGERY/OTHER,2009/12/31 00:00:00,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,6,2,7,1,NOT AVAILABLE
2,-9.785394e+06,3.580965e+06,3,00000645,CHRIST THE KING CATHOLIC SCHOOL,1503 MAIN ST,DAPHNE,AL,36526,NOT AVAILABLE,...,2009/12/10 00:00:00,IMAGERY/OTHER,2009/12/31 00:00:00,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,485,2,13,24,NOT AVAILABLE
3,-9.764468e+06,3.576040e+06,4,01604001,ST PATRICK CATHOLIC SCHOOL,23070 HWY 59 N,ROBERTSDALE,AL,36567,NOT AVAILABLE,...,2009/12/10 00:00:00,IMAGERY/OTHER,2009/12/31 00:00:00,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,156,2,13,17,NOT AVAILABLE
4,-9.786211e+06,3.581228e+06,5,A0500005,BAYSIDE ACADEMY,303 DRYER AVE,DAPHNE,AL,36526,NOT AVAILABLE,...,2009/12/10 00:00:00,IMAGERY/OTHER,2009/12/31 00:00:00,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,672,2,17,81,NOT AVAILABLE


2. Note that the data includes a FIPS county identifier. Such identifier is unique for each county in the United States. Currently, each observation of our dataset represents a county. I want to group our data by county and compute some summary statistics to be able to merge it with the SEDA data.

In [3]:
# keep only necessary variables
hilfd = hilfd[['NAME', 'STATE', 'ENROLLMENT', 'COUNTY', 'COUNTYFIPS']]
hilfd.head()

,NAME,STATE,ENROLLMENT,COUNTY,COUNTYFIPS
0,ST BENEDICT CATHOLIC SCHOOL,AL,151,BALDWIN,1003
1,VICTORY CHRISTIAN ACADEMY,AL,6,BALDWIN,1003
2,CHRIST THE KING CATHOLIC SCHOOL,AL,485,BALDWIN,1003
3,ST PATRICK CATHOLIC SCHOOL,AL,156,BALDWIN,1003
4,BAYSIDE ACADEMY,AL,672,BALDWIN,1003


In [4]:
# Groups data to obtain enrollment numbers and count of schools in county 
hilfd_grouped_schoolcounts = hilfd.groupby(by = ['COUNTYFIPS','COUNTY', 'STATE']).NAME.count().reset_index()
hilfd_grouped_enrollment = hilfd.groupby(by = 'COUNTYFIPS').ENROLLMENT.sum().reset_index()

# Merges so each county ID is mapped to both a total enrollment count and school count  
hilfd_county = hilfd_grouped_schoolcounts.merge(hilfd_grouped_enrollment, on = 'COUNTYFIPS')
hilfd_county = hilfd_county.rename(columns = {'NAME' : 'PRIV_SCHOOLS'})

hilfd_county.head()

,COUNTYFIPS,COUNTY,STATE,PRIV_SCHOOLS,ENROLLMENT
0,1001,AUTAUGA,AL,6,625
1,1003,BALDWIN,AL,9,1707
2,1005,BARBOUR,AL,2,346
3,1007,BIBB,AL,1,120
4,1009,BLOUNT,AL,2,135


3. Let's save this dataframe.

In [5]:
#Saving dataframe to csv
hilfd_county_clean_path = 'cleandata/hilfd_county_clean.csv'
hilfd_county.to_csv(path_or_buf = hilfd_county_clean_path, index = False)

### Part 2: Cleaning SEDA County Data to Prep for Merge

1. First load in our dataset from the csv I downloaded from the Educational Opportunity Project at Stanford University.

In [6]:
seda_raw_path = 'rawdata/seda_county_pool_cs_4.0.csv'
seda = pd.read_csv(seda_raw_path)
seda.head()

,sedacounty,sedacountyname,fips,stateabb,subcat,subgroup,gradecenter,gap,tot_asmts,cellcount,...,cs_mn_grd_ol_se,cs_mn_mth_ol_se,cs_mn_avg_eb,cs_mn_coh_eb,cs_mn_grd_eb,cs_mn_mth_eb,cs_mn_avg_eb_se,cs_mn_coh_eb_se,cs_mn_grd_eb_se,cs_mn_mth_eb_se
0,1001,Autauga County,1,AL,all,all,5.5,0,90120,120,...,0.006602,0.019285,0.018347,-0.021596,0.018256,-0.139268,0.009645,0.003298,0.006462,0.018923
1,1001,Autauga County,1,AL,race,asn,5.5,0,1274,99,...,NaN,NaN,0.430602,NaN,NaN,NaN,0.035202,NaN,NaN,NaN
2,1001,Autauga County,1,AL,race,blk,5.5,0,21564,120,...,0.007880,0.023220,-0.491160,-0.016851,0.008331,-0.130127,0.011592,0.003983,0.007694,0.022590
3,1001,Autauga County,1,AL,ecd,ecd,5.5,0,42350,120,...,0.007142,0.020828,-0.360849,-0.012469,0.006494,-0.130790,0.010398,0.003572,0.006981,0.020377
4,1001,Autauga County,1,AL,gender,fem,5.5,0,40006,108,...,0.007401,0.021432,0.072789,-0.021143,0.024315,-0.214264,0.010979,0.003827,0.007205,0.021025


2. The data provided here essentially has two types of test scores/gap/performance variables (the cs_... variables): ordinary least squares and empirical bayesian. The empirical bayesian estimates, according to the documentation provided from SEDA, are only to be used as predictor variables in a regression not as outcome (left-side variables) because they are shrunken estimates that might bias the coefficients. Since I am interested in using the performance variables as outcome variables, I will drop the (...eb) variables. 

In [7]:
seda_varlist = ['sedacounty', 'sedacountyname', 'stateabb', 'subgroup', 'gradecenter', 'gap', 'tot_asmts', 'mn_asmts', 
                'cs_mn_avg_ol', 'cs_mn_coh_ol', 'cs_mn_grd_ol', 'cs_mn_mth_ol', 'cs_mn_avg_ol_se', 'cs_mn_coh_ol_se', 'cs_mn_grd_ol_se', 'cs_mn_mth_ol_se']
seda_long = seda[seda_varlist]
seda_long.head()

,sedacounty,sedacountyname,stateabb,subgroup,gradecenter,gap,tot_asmts,mn_asmts,cs_mn_avg_ol,cs_mn_coh_ol,cs_mn_grd_ol,cs_mn_mth_ol,cs_mn_avg_ol_se,cs_mn_coh_ol_se,cs_mn_grd_ol_se,cs_mn_mth_ol_se
0,1001,Autauga County,AL,all,5.5,0,90120,751.000000,0.018759,-0.022213,0.019486,-0.145871,0.009654,0.003353,0.006602,0.019285
1,1001,Autauga County,AL,asn,5.5,0,1274,12.868687,0.435298,NaN,NaN,NaN,0.035566,NaN,NaN,NaN
2,1001,Autauga County,AL,blk,5.5,0,21564,179.700000,-0.491329,-0.017038,0.009625,-0.135009,0.011610,0.004034,0.007880,0.023220
3,1001,Autauga County,AL,ecd,5.5,0,42350,352.916667,-0.360748,-0.012836,0.007545,-0.137546,0.010415,0.003631,0.007142,0.020828
4,1001,Autauga County,AL,fem,5.5,0,40006,370.425926,0.073698,-0.022106,0.025230,-0.219353,0.011007,0.003913,0.007401,0.021432


3. If you look at our data, it has lots of missing values. For such a large dataset of such a range of counties, it makes sense that some might not have observations for every category. Since so many counties are missing one value or another, I have decided to leave in the NAs for cleaining. Omitting counties for missing one variable might bias results towards counties with more robust collection methods - or ones that meet criteria for having a certain value (e.g. having enough asian students to report their test scores). For future analyses and summaries, the appropriate counties will be omitted if they are missing the necessary variables, but as a a part of the data cleaning, I will not remove them.

4. The data I downloaded from SEDA is in long form, with a separate observation for each subgroup within a county rather than just the county (for a more detailed explanation of long vs wide data see this [explantion from Towards Data Science](https://towardsdatascience.com/reshape-pandas-dataframe-with-pivot-table-in-python-tutorial-and-visualization-2248c2012a31)). Having the data in both forms enables me to work with it differently, so I aim to generate two final forms of my dataset: one in long form and one in wide form.  

In [8]:
#Creating a wide form of our data
seda_wide = seda_long.pivot_table(index = ['sedacounty','sedacountyname','stateabb'], columns = 'subgroup', values = 
                                  ['tot_asmts', 'mn_asmts', 'cs_mn_avg_ol', 'cs_mn_coh_ol', 'cs_mn_grd_ol', 
                                   'cs_mn_mth_ol', 'cs_mn_avg_ol_se', 'cs_mn_coh_ol_se', 
                                   'cs_mn_grd_ol_se', 'cs_mn_mth_ol_se']).reset_index()
seda_wide.head()

sedacounty  sedacountyname stateabb cs_mn_avg_ol                      \
subgroup                                              all       asn       blk   
0              1001  Autauga County       AL     0.018759  0.435298 -0.491329   
1              1003  Baldwin County       AL    -0.006866  0.480640 -0.637915   
2              1005  Barbour County       AL    -0.593244 -0.046764 -0.810282   
3              1007     Bibb County       AL    -0.414999       NaN -0.805564   
4              1009   Blount County       AL    -0.224590  0.102954 -0.531608   

                                                  ... tot_asmts          \
subgroup       ecd       fem       hsp       mal  ...       mtr     nam   
0        -0.360748  0.073698  0.026954 -0.048415  ...    1384.0   109.0   
1        -0.345237  0.061267 -0.389110 -0.086230  ...    7887.0  1447.0   
2        -0.751294 -0.481684 -0.750132 -0.688593  ...     280.0     NaN   
3        -0.589172 -0.313002 -0.357055 -0.514083  ...     272.0     NaN   
4        -0.436760 -0.157655 -0.496297 -0.283575  ...     622.0   114.0   

                                                                      \
subgroup       nec       neg       wag       wbg       whg       wht   
0          47752.0   90102.0   53542.0   84684.0   60157.0   63120.0   
1         142456.0  269294.0  201120.0  236627.0  210599.0  207417.0   
2           8960.0   34470.0    8785.0   32613.0   12567.0   11898.0   
3          10845.0   31528.0    6724.0   30007.0   21641.0   23601.0   
4          40814.0   90187.0   44102.0   76410.0   87440.0   74893.0   

                              
subgroup       wmg       wng  
0          49537.0   30640.0  
1         174736.0  205335.0  
2           5928.0       NaN  
3          14685.0       NaN  
4          47987.0   38552.0  

[5 rows x 167 columns]

As you can see, my data is now VERY wide with 167 columns. Although such a wide dataset is a little cumbersome, it enables us to put our data in a form where each observation is just a county and perform analyses and visualizations on that. Additionally, we now have a hierarchical column index. Our super columns are our original outcome variables, and our sub columns are the value of that for each subgroup.

5. Now let's save both the long and short forms of our data.

In [9]:
#save long data
seda_long_clean_path = 'seda_long_clean.csv'
seda_long.to_csv(seda_long_clean_path)

#save wide data
seda_wide_clean_path = 'seda_wide_clean.csv'
seda_wide.to_csv(seda_wide_clean_path)

### Part 3: Cleaining SEDA Covariates For Merge or Future Use

Alongside the primary data on performance, SEDA also offers county level covariates of various stripes. Some of these could be useful controls for our predictive models, so I want to add these in as well. 

1. Let's load in the dataset

In [10]:
# reads in the csv from HIFLD
seda_cov_raw_path = 'seda_cov_county_pool_4.0.csv'
seda_cov = pd.read_csv(seda_cov_raw_path)
seda_cov.columns

Index(['sedacounty', 'sedacountyname', 'fips', 'urban', 'suburb', 'town',
       'rural', 'perind', 'perasn', 'perhsp', 'perblk', 'perwht', 'perfl',
       'perrl', 'perfrl', 'totenrl', 'hswhtblk', 'hswhthsp', 'hsflnfl',
       'hsecdnec', 'rswhtblk', 'rswhthsp', 'rsflnfl', 'rsecdnec', 'perecd',
       'sesavgall', 'sesavgasn', 'sesavgblk', 'sesavghsp', 'sesavgnam',
       'sesavgwht', 'sesavgwhtblk', 'sesavgwhthsp', 'lninc50avgall',
       'baplusavgall', 'unempavgall', 'snapavgall', 'povertyavgall',
       'single_momavgall', 'lninc50avgblk', 'baplusavgblk', 'unempavgblk',
       'snapavgblk', 'povertyavgblk', 'single_momavgblk', 'lninc50avghsp',
       'baplusavghsp', 'unempavghsp', 'snapavghsp', 'povertyavghsp',
       'single_momavghsp', 'lninc50avgwht', 'baplusavgwht', 'unempavgwht',
       'snapavgwht', 'povertyavgwht', 'single_momavgwht'],
      dtype='object')

2. Looking at this data, there are a lot of covariates, and at this stage in our process, I'm not entirely sure which ones we want to include in our model later. The data is already pretty clean, so I'll just add some code we can fill in later to make sure we get the right covariates. For now, I will just leave in the percent economically disadvantaged and socio economic status measure for everyone in the county.

In [11]:
#keeps only covariates of interest

#fill in list with the covariates of interest for later
seda_cov_varlist = ['sedacounty', 'sedacountyname'] + ['perecd', 'sesavgall']
seda_cov = seda_cov[seda_cov_varlist]
seda_cov.head()

,sedacounty,sedacountyname,perecd,sesavgall
0,1001,Autauga County,0.470157,0.130665
1,1003,Baldwin County,0.474722,0.390299
2,1005,Barbour County,0.759069,-1.591195
3,1007,Bibb County,0.656081,-0.226344
4,1009,Blount County,0.548490,0.082366


3. Now we can save the data for future use.

In [12]:
#saves dataframe to csv
seda_cov_clean_path = 'seda_cov_clean.csv'
seda_cov.to_csv(path_or_buf = seda_cov_clean_path)